In [1]:
import pymysql
import json
import pandas as pd
conn = pymysql.connect(host = '120.55.167.182',user = 'root',password = 'wda20190707',database = 'dota')
cursor = conn.cursor()

sql = '''
select player_1_hero_id,player_2_hero_id,player_3_hero_id,
       player_4_hero_id,player_5_hero_id,player_6_hero_id,player_7_hero_id
    ,player_8_hero_id,player_9_hero_id,player_10_hero_id,win
from dota.`match`
where (game_mode = 22 or game_mode = 3) and 
    not (player_1_hero_id = 'None'
   or player_2_hero_id ='None'
   or player_3_hero_id = 'None'
   or player_4_hero_id ='None'
   or player_5_hero_id = 'None'
   or player_6_hero_id ='None'
   or player_7_hero_id = 'None'
   or player_8_hero_id ='None'
   or player_9_hero_id = 'None'
   or player_10_hero_id ='None')

'''
cursor.execute(sql)
match_data = cursor.fetchall()
match_data = list(match_data)
match_data = list(map(list,match_data))
match_df = pd.DataFrame(match_data)
columns = []
for i in range(1,11):
    columns.append("player_"+str(i)+"_hero_id")
columns.append("target")
match_df.columns = columns

hero_stats = pd.read_csv("Hero2Vector/input/hero_stats.csv")
print(hero_stats.columns)

kz,hx,fz,ts,bf,xs,nj,tj = {},{},{},{},{},{},{},{}
for index,row in hero_stats.iterrows():
    kz[row['id']] = row['kong_zhi']
    hx[row['id']] = row['he_xin']
    fz[row['id']] = row['fu_zhu']
    ts[row['id']] = row['tao_sheng']
    bf[row['id']] = row['bao_fa']
    xs[row['id']] = row['xian_shou']
    nj[row['id']] = row['nai_jiu']
    tj[row['id']] = row['tui_jin']

for i in range(8):
    match_df['radiant_$' + str(i)] = 0
    match_df['dire_$' + str(i)] = 0

print(match_df.head(5))



Index(['Unnamed: 0', 'hero_name', 'id', 'pro_pick', 'pro_ban', 'pro_win',
       'he_xin', 'xian_shou', 'fu_zhu', 'bao_fa', 'kong_zhi', 'nai_jiu',
       'tui_jin', 'tao_sheng'],
      dtype='object')
  player_1_hero_id player_2_hero_id player_3_hero_id player_4_hero_id  \
0               83               34               63              106   
1               44              110               11               50   
2                6              110               45               18   
3               31               59               23               44   
4               27               96               74               93   

  player_5_hero_id player_6_hero_id player_7_hero_id player_8_hero_id  \
0               50               99                4                7   
1               14               99                8               26   
2               14               36               74               31   
3               57               12              114               1

In [2]:
# 检查英雄ID为None的数据是否存在
for col in match_df.columns:
    if col.startswith("player"):
        print(col)
        print(len(match_df[match_df[col]=='None']))


player_1_hero_id
0
player_2_hero_id
0
player_3_hero_id
0
player_4_hero_id
0
player_5_hero_id
0
player_6_hero_id
0
player_7_hero_id
0
player_8_hero_id
0
player_9_hero_id
0
player_10_hero_id
0


In [3]:
# 分别计算天辉和夜宴阵容的8维能力维度总和
# 即 控制、核心、爆发、推进、辅助、先手、耐久、逃生
# （偷懒用拼音了嘻嘻）
j = 0
for dic in kz,hx,fz,ts,bf,xs,nj,tj:


        # 产生8*2列 夜宴天辉分别8列
        for i in range(5):
            #print(match_df['radiant_$'+str(j)])
            match_df['radiant_$'+str(j)] = match_df['radiant_$'+str(j)]+match_df["player_" + str(i + 1) + "_hero_id"].apply(lambda x: int(dic[int(x)]))

        for i in range(5,10):
            #print(match_df['dire_$' + str(j)])
            match_df['dire_$' + str(j)] = match_df['dire_$' + str(j)]+match_df["player_" + str(i + 1) + "_hero_id"].apply(lambda x: int(dic[int(x)]))
        j = j + 1

In [4]:
import plotly.express as px
# ployly 绘图库

In [5]:
# dataexplore 阶段 
# 分析观察各个维度与胜负的关系情况


# 基本信息
# match_df.columns

# Index(['player_1_hero_id', 'player_2_hero_id', 'player_3_hero_id',
#        'player_4_hero_id', 'player_5_hero_id', 'player_6_hero_id',
#        'player_7_hero_id', 'player_8_hero_id', 'player_9_hero_id',
#        'player_10_hero_id', 'target', 'radiant_kongzhi', 'dire_kongzhi',
#        'radiant_hexin', 'dire_hexin', 'radiant_fuzhu', 'dire_fuzhu',
#        'radiant_taosheng', 'dire_taosheng', 'radiant_baofa', 'dire_baofa',
#        'radiant_xianshou', 'dire_xianshou', 'radiant_naijiu', 'dire_naijiu',
#        'radiant_tuijin', 'dire_tuijin'],
#       dtype='object')

# 264367
# len(match_df)

In [6]:
# 修改列名为了方便
columns = ['player_1_hero_id', 'player_2_hero_id', 'player_3_hero_id',
       'player_4_hero_id', 'player_5_hero_id', 'player_6_hero_id',
       'player_7_hero_id', 'player_8_hero_id', 'player_9_hero_id',
       'player_10_hero_id', 'target', 'radiant_kongzhi', 'dire_kongzhi', 'radiant_hexin',
       'dire_hexin', 'radiant_fuzhu', 'dire_fuzhu', 'radiant_taosheng', 'dire_taosheng',
       'radiant_baofa', 'dire_baofa', 'radiant_xianshou', 'dire_xianshou', 'radiant_naijiu',
       'dire_naijiu', 'radiant_tuijin', 'dire_tuijin']

match_df.columns = columns

In [7]:
match_df['kongzhi_diff'] = match_df['radiant_kongzhi']-match_df['dire_kongzhi']
match_df['taosheng_diff'] = match_df['radiant_taosheng']-match_df['dire_taosheng']
match_df['hexin_diff'] = match_df['radiant_hexin']-match_df['dire_hexin']
match_df['naijiu_diff'] = match_df['radiant_naijiu']-match_df['dire_naijiu']
match_df['xianshou_diff'] = match_df['radiant_xianshou']-match_df['dire_xianshou']
match_df['baofa_diff'] = match_df['radiant_baofa']-match_df['dire_baofa']
match_df['tuijin_diff'] = match_df['radiant_tuijin']-match_df['dire_tuijin']
match_df['fuzhu_diff'] = match_df['radiant_fuzhu']-match_df['dire_fuzhu']

In [8]:
match_df.head(5)

,player_1_hero_id,player_2_hero_id,player_3_hero_id,player_4_hero_id,player_5_hero_id,player_6_hero_id,player_7_hero_id,player_8_hero_id,player_9_hero_id,player_10_hero_id,...,radiant_tuijin,dire_tuijin,kongzhi_diff,taosheng_diff,hexin_diff,naijiu_diff,xianshou_diff,baofa_diff,tuijin_diff,fuzhu_diff
0,83,34,63,106,50,99,4,7,14,21,...,2,0,-3,6,1,-3,-4,0,2,4
1,44,110,11,50,14,99,8,26,95,119,...,0,3,0,0,-2,-1,1,3,-3,0
2,6,110,45,18,14,36,74,31,8,71,...,3,1,1,-1,-1,1,4,0,2,-2
3,31,59,23,44,57,12,114,14,7,33,...,0,3,-5,-3,2,3,-5,1,-3,5
4,27,96,74,93,68,28,31,1,121,44,...,3,0,5,-1,-5,1,2,3,3,0


In [9]:
df = match_df[['target', 'radiant_kongzhi', 'dire_kongzhi',
       'radiant_hexin', 'dire_hexin', 'radiant_fuzhu', 'dire_fuzhu',
       'radiant_taosheng', 'dire_taosheng', 'radiant_baofa', 'dire_baofa',
       'radiant_xianshou', 'dire_xianshou', 'radiant_naijiu', 'dire_naijiu',
       'radiant_tuijin', 'dire_tuijin', 'kongzhi_diff', 'taosheng_diff',
       'hexin_diff', 'naijiu_diff', 'xianshou_diff', 'baofa_diff',
       'tuijin_diff', 'fuzhu_diff']]

In [10]:
df.head(5)

,target,radiant_kongzhi,dire_kongzhi,radiant_hexin,dire_hexin,radiant_fuzhu,dire_fuzhu,radiant_taosheng,dire_taosheng,radiant_baofa,...,radiant_tuijin,dire_tuijin,kongzhi_diff,taosheng_diff,hexin_diff,naijiu_diff,xianshou_diff,baofa_diff,tuijin_diff,fuzhu_diff
0,0,3,6,5,4,6,2,7,1,5,...,2,0,-3,6,1,-3,-4,0,2,4
1,0,4,4,5,7,4,4,3,3,8,...,0,3,0,0,-2,-1,1,3,-3,0
2,0,6,5,4,5,1,3,2,3,7,...,3,1,1,-1,-1,1,4,0,2,-2
3,0,2,7,7,5,6,1,1,4,4,...,0,3,-5,-3,2,3,-5,1,-3,5
4,0,8,3,3,8,4,4,5,6,8,...,3,0,5,-1,-5,1,2,3,3,0


In [11]:
dimension = ['kongzhi','hexin','fuzhu','baofa','taosheng','naijiu','xianshou','tuijin']

In [12]:
df['radiant_average'] = 0
df['dire_average'] = 0
for d in dimension:
    df['radiant_average'] = df['radiant_average']+df['radiant_'+d]
    df['dire_average'] = df['dire_average']+df['dire_'+d]


c:\users\wda\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\users\wda\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\users\wda\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [13]:
df.columns

Index(['target', 'radiant_kongzhi', 'dire_kongzhi', 'radiant_hexin',
       'dire_hexin', 'radiant_fuzhu', 'dire_fuzhu', 'radiant_taosheng',
       'dire_taosheng', 'radiant_baofa', 'dire_baofa', 'radiant_xianshou',
       'dire_xianshou', 'radiant_naijiu', 'dire_naijiu', 'radiant_tuijin',
       'dire_tuijin', 'kongzhi_diff', 'taosheng_diff', 'hexin_diff',
       'naijiu_diff', 'xianshou_diff', 'baofa_diff', 'tuijin_diff',
       'fuzhu_diff', 'radiant_average', 'dire_average'],
      dtype='object')

In [14]:
# 总维度值差超过5的占据总数据量一半,而根据该差值
test_df = df[(abs(df['dire_average']-df['radiant_average']))>8]

In [15]:
# target = 0 ：18900
# target = 1 ：22000
test_df[test_df['target']=="0"]
test_df[(test_df['target']=="0") & (test_df['dire_average']-test_df['radiant_average']<-8)]

,target,radiant_kongzhi,dire_kongzhi,radiant_hexin,dire_hexin,radiant_fuzhu,dire_fuzhu,radiant_taosheng,dire_taosheng,radiant_baofa,...,kongzhi_diff,taosheng_diff,hexin_diff,naijiu_diff,xianshou_diff,baofa_diff,tuijin_diff,fuzhu_diff,radiant_average,dire_average
22,0,5,4,7,6,3,0,6,1,7,...,1,5,1,-1,1,0,4,3,38,24
146,0,8,3,6,5,5,4,5,5,9,...,5,0,1,-1,3,4,1,1,42,28
161,0,8,5,5,5,4,3,3,3,7,...,3,0,0,-3,4,5,-1,1,37,28
194,0,7,6,5,3,3,7,5,3,9,...,1,2,2,3,0,1,6,-4,41,30
207,0,9,5,4,4,6,4,5,3,8,...,4,2,0,-4,3,2,3,2,40,28
226,0,8,4,7,3,3,5,3,1,6,...,4,2,4,2,3,-2,-2,-2,37,28
227,0,7,6,6,2,3,5,2,2,6,...,1,0,4,0,4,1,3,-2,39,28
392,0,7,6,7,5,5,1,5,4,5,...,1,1,2,-3,0,1,5,4,41,30
417,0,5,4,6,5,2,4,7,1,7,...,1,6,1,-3,-2,4,4,-2,36,27
483,0,6,5,3,6,4,2,3,2,9,...,1,1,-3,-2,5,5,0,2,37,28


In [16]:
test_df['kongzhi_diff'].value_counts()

 2     6039
-2     5830
 3     5739
 1     5682
-1     5677
-3     5671
 0     5552
 4     4674
-4     4532
-5     3060
 5     3040
 6     1642
-6     1642
 7      701
-7      680
-8      219
 8      197
-9       47
 9       46
-10       9
 10       5
-11       1
 11       1
Name: kongzhi_diff, dtype: int64

In [17]:
import sklearn

In [19]:
relation_df = pd.read_sql("select * from dota.hero_relationship",conn)

In [20]:
relation_df.head(5)

,hero_id,target_hero_id,match_count,win_rate,update_ymd
0,1,16,170,0.48,2020-01-26
1,1,55,145,0.45,2020-01-26
2,1,52,137,0.44,2020-01-26
3,1,8,115,0.45,2020-01-26
4,1,106,107,0.49,2020-01-26


In [ ]:
def f(hero_1,hero_6,hero_7,hero_8,hero_9,hero_10):
    sum_win_rate = 0
    l = relation_df[ (match_df['player_1_hero_id']==hero_1) & (match_df['player_6_hero_id'] == hero_6)]['win_rate'].values.tolist()
    if(len(l)==0):
        tmp = 0
    else:
        tmp = l[0]
    sum_win_rate = sum_win_rate + tmp
    return sum_win_rate


match_df['sum_win_rate_relative'] = match_df.apply(
    lambda x:f(x.player_1_hero_id,x.player_6_hero_id,x.player_7_hero_id,x.player_8_hero_id,x.player_9_hero_id,x.player_10_hero_id)
    +f(x.player_2_hero_id,x.player_6_hero_id,x.player_7_hero_id,x.player_8_hero_id,x.player_9_hero_id,x.player_10_hero_id)
    +f(x.player_3_hero_id,x.player_6_hero_id,x.player_7_hero_id,x.player_8_hero_id,x.player_9_hero_id,x.player_10_hero_id)
    +f(x.player_4_hero_id,x.player_6_hero_id,x.player_7_hero_id,x.player_8_hero_id,x.player_9_hero_id,x.player_10_hero_id)
    +f(x.player_5_hero_id,x.player_6_hero_id,x.player_7_hero_id,x.player_8_hero_id,x.player_9_hero_id,x.player_10_hero_id)
    ,axis = 1)



c:\users\wda\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

